In [1]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import csv
import pickle

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn import preprocessing
from autokeras.image.image_supervised import load_image_dataset

from autokeras.image.image_supervised import ImageClassifier
from autokeras.image.image_supervised import load_image_dataset
import keras
import numpy as np
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
 


from IPython.display import display
pd.set_option('display.max_rows',5)
pd.set_option('precision',3)

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important;}</style>")

/Users/lovelife/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/lovelife/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following

In [2]:
all_train_images=os.listdir("./data/box/images")
#print(len(all_train_images))
labels=[x.split("_")[0] for x in all_train_images]
labels_my=[int(x.split("_")[1].split('.png')[0]) for x in all_train_images]
labels_my

# trail all, delete

filt=pd.DataFrame({'image':all_train_images,'label':labels,'num':labels_my})
filt
all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()

with open('train_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()

x_train, y_train = load_image_dataset(csv_file_path="train_label.csv",
                                      images_path="./data/box/images")
print(x_train.shape)
print(y_train.shape)


filt = pd.read_csv('./val/val_annotations.txt', sep="\t", header=None)
filt.columns=['image','label','a','b','c','d']
filt['image']=filt['image'].apply(lambda x: x.split('.')[0]+'.png')

all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()

with open('valid_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()
    
x_valid, y_valid = load_image_dataset(csv_file_path="valid_label.csv",
                                      images_path="./data/val/box/images")
print(x_valid.shape)
print(y_valid.shape)


all_train_images=os.listdir("./data/test/box/images")
labels=['unknow' for x in all_train_images]
labels_my=['unknow' for x in all_train_images]


filt=pd.DataFrame({'image':all_train_images,'label':labels,'num':labels_my})
all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()
import csv
with open('test_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()
    
x_test, y_test = load_image_dataset(csv_file_path="test_label.csv",
                                      images_path="./data/test/box/images")
print(x_test.shape)

x_train = x_train.astype("float") / 255.0
x_valid = x_valid.astype("float") / 255.0
x_test = x_test.astype("float") / 255.0
TRAINING_TIMES = [
    60 * 60 *6, # 6 hour
    60 * 60 * 2, # 2 hours
    60 * 60 * 3, # 3 hours
    60 * 60 * 4, # 4 hours
]

(100000, 32, 32, 3)
(100000,)
(10000, 32, 32, 3)
(10000,)
(10000, 32, 32, 3)


In [6]:
results=[]
for seconds in TRAINING_TIMES:
    print("[INFO] training model for {} seconds max...".format(seconds))
    clf = ImageClassifier(verbose=True,augment=True, searcher_args={'trainer_args':{'max_no_improvement_num':4}})
    clf.fit(x_train, y_train, time_limit=seconds)
    clf.final_fit(x_train, y_train,x_valid, y_valid, retrain=True)
results

[INFO] training model for 360.0 seconds max...
Saving Directory: /var/folders/h1/pcv8265963799lkgn7tjtctw0000gn/T/autokeras_0O5MCV
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
Epoch-2, Current Metric - 0.032:  21%|████▉                   | 160/778 [00:58<04:00,  2.57 batch/s]

TimeoutError: Search Time too short. No model was found during the search time.

In [17]:
# Score trained model.
scores = clf.evaluate(x_train, y_train)
print('Train loss:', scores[0])
print('Train accuracy:', scores[1])
scores = clf.evaluate(x_valid, y_valid)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])

100000/100000 [==============================] - 25s 246us/step
Train loss: 1.141592525024414
Train accuracy: 0.70272
10000/10000 [==============================] - 2s 247us/step
Validation loss: 3.2589765857696533
Validation accuracy: 0.31


In [0]:
import csv
y_predict_p2=clf.predict(x_test)
with open('y_predicted_p2.csv', 'w') as train_csv:
    fieldnames = ['Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(y_predict_p2)):
        writer.writerow({ 'Label':y_predict_p2[i]})
    train_csv.close()